<a href="https://colab.research.google.com/github/AnnSenina/python_hse_2024/blob/main/notebooks/12_%D0%9C%D0%BE%D0%B4%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D0%B8_linreg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Усложняем линейную регрессию

Загрузите набор данных sandler.csv - набор данных для предсказания дохода от показа фильмов.

Столбцы Date - дата премьеры, Title - название, Genre - жанр, Studio - студия Theaters - число кинотеатров, Opening Gross - доход от премьеры, Opening Theaters - число кинотеатров для премьеры, Gross - общий доход


In [1]:
# библиотеки
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np

# новые импорты
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/Other/main/sandler.csv')
df.head()

In [ ]:
# сделайте предобработку - она уже прописана
df['Opening Theaters']=df['Opening Theaters'].apply(lambda x: x.replace(' ','',10).replace("$",'',10))
df['Theaters']=df['Theaters'].apply(lambda x: x.replace(' ','',10).replace("$",'',10))
df['Gross']=df['Gross'].apply(lambda x: x.replace(' ','',10).replace("$",'',10))
df['Opening Gross']=df['Opening Gross'].apply(lambda x: x.replace(' ','',10).replace("$",'',10))
df.drop(['Date','Title', 'Genre', 'Studio'], axis=1, inplace=True)
df['Opening Theaters'] = pd.to_numeric( df['Opening Theaters'])
df['Theaters'] = pd.to_numeric( df['Theaters'])
df['Gross'] = pd.to_numeric( df['Gross'])
df['Opening Gross'] = pd.to_numeric( df['Opening Gross'])

In [ ]:
df.head()

In [ ]:
sns.heatmap(df.corr(), cmap='RdYlGn', center=0, annot=True);
# вывод: коллинеарность

In [ ]:
sns.pairplot(df);
# вывод2: для некоторых переменных кривая, возможно, подойдет больше прямой линии

# Регуляризация

In [43]:
x = df[['Theaters', 'Opening Gross', 'Opening Theaters']] # добавили показатели
y = df['Gross']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

In [44]:
model = LinearRegression().fit(x_train, y_train)
y_pred = model.predict(x_test)

In [ ]:
r2 = metrics.r2_score(y_test, y_pred)
meanAbErr = metrics.mean_absolute_error(y_test, y_pred)
meanSqErr = metrics.mean_squared_error(y_test, y_pred)
print('R squared:', r2)
print('Mean Absolute Error:', meanAbErr)
print('Mean Square Error:', meanSqErr)

Можно ли как-то сохранить этот высокий R-квадрат, сделав поправку на коллианерность?

Регуляризация: изменение наклона линии путём ограничения весов модели, более высокое смещение корректируется за счет более низкой дисперсии (R в квадрате)

Мы должны добавить к функции потерь параметр регуляризации, который будет штрафовать модель за величину коэффициентов.
Чем больше параметр регуляризации, тем больше модель штрафуется за величину коэффициентов и их количество.
В хорошей модели у релевантных признаков, хорошо объясняющих зависимую переменную, должны быть коэффициенты больше, чем у незначимых признаков.

**Lasso** имеет более выраженную тенденцию к занулению коэффициентов (=избавлению от признаков). Она может быть полезна, если вы:
- Заведомо знаете, что не все признаки будут вам полезны;
- Имеете ограничения по скорости построения предсказаний, и вам выгодно избавляться от “лишних” признаков;
- Имеете выборку, где объектов меньше, чем признаков.

**Гребневая регрессия** не зануляет коэффициенты, а скорее старается уменьшить слишком большие (сильнее всего занижает веса для признаков с высокой корреляцией). Этот метод подходит, если вы уверены, что все ваши независимые переменные будут иметь эффект на независимую, пусть небольшой.

**Эластичная сеть** представляет собой комбинацию первых двух: может принести особую пользу в ситуациях, когда в данных необходимо одновременно выполнять отбор признаков и бороться с мультиколлинеарностью.


In [ ]:
sk_lasso_regression = Lasso()
sk_lasso_regression.fit(x_train, y_train)

sk_lasso_pred_res = sk_lasso_regression.predict(x_test)

Lasso_meanAbErr = metrics.mean_absolute_error(y_test, sk_lasso_pred_res)
Lasso_meanSqErr = metrics.mean_squared_error(y_test, sk_lasso_pred_res)
Lasso_r_score = metrics.r2_score(y_test, sk_lasso_pred_res)

print(f'Lasso Mean Square Error: {Lasso_meanSqErr}')
print(f'Lasso Mean Absolute Error: {Lasso_meanAbErr}')
print(f'Lasso R2 score: {Lasso_r_score}')

In [ ]:
sk_ridge_regression = Ridge()
sk_ridge_regression.fit(x_train, y_train)

sk_ridge_pred_res = sk_ridge_regression.predict(x_test)

ridge_meanAbErr = metrics.mean_absolute_error(y_test, sk_ridge_pred_res)
ridge_meanSqErr = metrics.mean_squared_error(y_test, sk_ridge_pred_res)
ridge_r_score = metrics.r2_score(y_test, sk_ridge_pred_res)

print(f'Ridge Mean Square Error: {ridge_meanSqErr}')
print(f'Ridge Mean Absolute Error: {ridge_meanAbErr}')
print(f'Ridge R2 score: {ridge_r_score}')

In [ ]:
sk_elastic_net_regression = ElasticNet()
sk_elastic_net_regression.fit(x_train, y_train)

sk_elastic_net_pred_res = sk_elastic_net_regression.predict(x_test)

EN_meanAbErr = metrics.mean_absolute_error(y_test, sk_elastic_net_pred_res)
EN_meanSqErr = metrics.mean_squared_error(y_test, sk_elastic_net_pred_res)
EN_r_score = metrics.r2_score(y_test, sk_elastic_net_pred_res)
print(f'Ridge Mean Square Error: {EN_meanSqErr}')
print(f'Ridge Mean Absolute Error: {EN_meanAbErr}')
print(f'Ridge R2 score: {EN_r_score}')

In [13]:
# бонус к кинотератам: преобразование данных может избавить от мультиколлинеарности и улучшить точность прогнозирования

b = df['Opening Theaters'] / df['Theaters']  # доля театров, участвовавших в премьере
c = df['Opening Gross'] / df['Opening Theaters']  # доход на 1 театр, участвовавший в премьере

x = [[b, c]]
# Дарья Гордеева

# Полиномиальная регрессия

In [ ]:
# сначала линейная регрессия с прошлой пары
# функция линейной регрессии Y = a + bX

x = df['Theaters']
y = df["Gross"]
res = scipy.stats.linregress(x, y)
print(res.rvalue**2)

In [ ]:
# функция линейной регрессии Y = a + bX

plt.plot(x, y, 'o', label='оригинальные данные')
plt.plot(x, res.intercept + res.slope*x, 'r', label='линия регрессии')
plt.legend();
# intercept - число, которое мы прибавляем в уравнении
# slope - коэффициент для х

В полиномиальной регрессии связь между независимой переменной x и зависимой переменной y моделируется как полином n-й степени от x.

Полиномиальная регрессия аналогична линейной регрессии, за исключением того, что в уравнении используются разные степени x (x, x², x³…), в отличие от линейной регрессии, в которой используется только x.



Документация: [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html), [Polynomial and Spline interpolation¶](https://scikit-learn.org/stable/auto_examples/linear_model/plot_polynomial_interpolation.html#sphx-glr-auto-examples-linear-model-plot-polynomial-interpolation-py)

In [ ]:
X = df[['Theaters']].values
y = df['Gross'].values

regr = LinearRegression()

quadratic = PolynomialFeatures(degree=2) # степени
X_quad = quadratic.fit_transform(X)

# fit features
X_fit = np.arange(X.min(), X.max(), 1)[:, np.newaxis]

regr = regr.fit(X_quad, y)
y_quad_fit = regr.predict(quadratic.fit_transform(X_fit))
quadratic_r2 = metrics.r2_score(y, regr.predict(X_quad))


# plot results
plt.scatter(X, y, color='lightgray')

plt.plot(X_fit, y_quad_fit,
         label='quadratic (d=2), $R^2={:.2f}$'.format(quadratic_r2),
         color='red',
         lw=2,
         linestyle='-')

plt.xlabel('Количество кинотеатров')
plt.ylabel('Общий доход')
plt.legend(loc='lower right');

Внимание, вопрос: стало лучше или хуже? Как понять, нужно ли искривлять линию дальше?

In [ ]:
# посмотрим на ошибки при увеличении степени
number_degrees = range(1, 11)
plt_mean_squared_error = []
for degree in number_degrees:

   poly_model = PolynomialFeatures(degree=degree)

   poly_x_values = poly_model.fit_transform(X)
   poly_model.fit(poly_x_values, y)

   regression_model = LinearRegression()
   regression_model.fit(poly_x_values, y)
   y_pred = regression_model.predict(poly_x_values)

   plt_mean_squared_error.append(metrics.mean_squared_error(y, y_pred, squared=False))

plt.scatter(number_degrees,plt_mean_squared_error, color="green")
plt.plot(number_degrees,plt_mean_squared_error, color="red");

# Практика

- подберите подходящую степень для x
- обучите модель


In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/Other/main/Ice_cream%20selling%20data.csv')
df.head()

In [ ]:
x = df['Temperature (°C)']
y = df["Ice Cream Sales (units)"]
res = scipy.stats.linregress(x, y)
print(res.rvalue**2)

In [ ]:
# функция линейной регрессии Y = a + bX

plt.plot(x, y, 'o', label='оригинальные данные')
plt.plot(x, res.intercept + res.slope*x, 'r', label='линия регрессии')
plt.legend();

In [22]:
# ваш код



Возможное решение

In [ ]:
# @title
X = df[['Temperature (°C)']].values
y = df['Ice Cream Sales (units)'].values

regr = LinearRegression()

quadratic = PolynomialFeatures(degree=4) # степени
X_quad = quadratic.fit_transform(X)

# fit features
X_fit = np.arange(X.min(), X.max(), 1)[:, np.newaxis]

regr = regr.fit(X_quad, y)
y_quad_fit = regr.predict(quadratic.fit_transform(X_fit))
quadratic_r2 = metrics.r2_score(y, regr.predict(X_quad))


# plot results
plt.scatter(X, y, color='lightgray')

plt.plot(X_fit, y_quad_fit,
         label='d=4, $R^2={:.2f}$'.format(quadratic_r2),
         color='red',
         lw=2,
         linestyle='-')

plt.xlabel('Количество кинотеатров')
plt.ylabel('Общий доход')
plt.legend(loc='lower right');

In [ ]:
# @title
# посмотрим на ошибки при увеличении степени
number_degrees = range(1, 11)
plt_mean_squared_error = []
for degree in number_degrees:

   poly_model = PolynomialFeatures(degree=degree)

   poly_x_values = poly_model.fit_transform(X)
   poly_model.fit(poly_x_values, y)

   regression_model = LinearRegression()
   regression_model.fit(poly_x_values, y)
   y_pred = regression_model.predict(poly_x_values)

   plt_mean_squared_error.append(metrics.mean_squared_error(y, y_pred, squared=False))

plt.scatter(number_degrees,plt_mean_squared_error, color="green")
plt.plot(number_degrees,plt_mean_squared_error, color="red");

# например, возьмем 4 степень